In [ ]:
import torch
import utils

ModuleNotFoundError: No module named 'utils'

In [3]:
opt = {
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "epochs": 500,
    "d_updates": 1,
    "plot_frequency": 10,
    "lr": 0.0001,
    "latent_size": 32,
    "samples": 10000,
    "batch_size": 500,
    "loss": "standard",  # Options: 'wasserstein', 'wasserstein-gp'
    "spectral_norm": False,
    "clip_weights": 0.0,  # Set to a positive value to enable weight clipping
    "topk": False,  # Enables top-k training after half of the epochs
}